In [ ]:
from Organizer import Organizer
from search_agent import Research_Tool
import random
LENGTHY_SEARCH_PROMPT = """
                            Answer this question in this format:
                            {"status":"pending", "urls": list(str)}
                            YOU MUST PUT THE JSON ANSWER WITHIN TWO ```

                            Return a list of upto 5 urls from the provided ones that you would like to visit for more Context to answer the following question.
                            Question:
                            
                        """


Researcher = Organizer()
search = Research_Tool()

query = "Tesla by elon musk"
urls = search.search(query, random.randint(10,15))
print(urls)
data = Researcher.get_filtered_urls(urls, LENGTHY_SEARCH_PROMPT, query)
print(data)
processed_data = await Researcher.process_url(data)
final_report = await Researcher.multi_text_processor(query)


final_report

In [ ]:
from bs4 import BeautifulSoup
import jsbeautifier
from urllib.parse import urlparse, urljoin, quote_plus
from typing import Dict, Any, List
import requests


DEBUG = True

def log_debug(message):
    if DEBUG:
        print(f"DEBUG: {message}")

headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,/;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Referer': 'https://www.google.com/',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
        }


def _perform_web_search(query: str) -> List[Dict[str, Any]]:
    encoded_query = quote_plus(query)
    search_url = f"https://www.google.com/search?q={encoded_query}&tbm=vid"
    log_debug(f"Search URL: {search_url}")
    
    try:
        log_debug("Sending GET request to Google")
        response = requests.get(search_url, headers=headers, timeout=5)
        log_debug(f"Response status code: {response.status_code}")
        response.raise_for_status()
        
        log_debug("Parsing HTML with BeautifulSoup")
        soup = BeautifulSoup(response.text, 'html.parser')
        print(soup)
        # log_debug("Searching for result divs")
        # search_results = []
        # for g in soup.find_all('div', class_='g'):
        #     log_debug("Processing a search result div")
        #     anchor = g.find('a')
        #     title = g.find('h3').text if g.find('h3') else 'No title'
        #     url = anchor.get('href', 'No URL') if anchor else 'No URL'
            
        #     description = ''
        #     description_div = g.find('div', class_=['VwiC3b', 'yXK7lf'])
        #     if description_div:
        #         description = description_div.get_text(strip=True)
        #     else:
        #         description = g.get_text(strip=True)
            
        #     log_debug(f"Found result: Title: {title[:30]}..., URL: {url[:30]}...")
        #     search_results.append({
        #         'title': title,
        #         'description': description,
        #         'url': url
        #     })
        
        # log_debug(f"Successfully retrieved {len(search_results)} search results for query: {query}")
        # return search_results
    except requests.RequestException as e:
        log_debug(f"Error performing search: {str(e)}")
        return []

_perform_web_search("robovan")

In [ ]:
print(final_report)

In [ ]:
import requests
from utils import split_corpus
import concurrent.futures
import openai
import json
import re

query = """Generate me a report within 500 words in JSON format using this schema:
            {"summery": list({"heading": sescription}, ...), "Images": list({"url": str, "description": str}, ...), "links": list({"url": str, "description": str}, ...)}
            The Images and the Links Must be Present in the context. if no Image or Link found keep the list empty.
            DO NOT USE YOUR KNOWLEDGE TO CONSTRUCT THE ANSWER. USE ONLY WHAT IS PROVIDED IN THE CONTEXT.
            add only the image and links that are strongly related to the report topic.
            in the summery describe the Report topic based on the provided context briefly within 800 words.
            Report Topic:
            Elon Musk"""
res = requests.get("https://en.wikipedia.org/wiki/Elon_Musk")

data = split_corpus(res.text, max_words=1350)
api_keys = open("keys.txt").read().split("\n")

print(len(data))

def extract_query(text: str) -> str:
    pattern = r"```(.*?)```"
    matches = re.findall(pattern, text, re.DOTALL)
    return matches[0] if matches else text

def ask_llm(data):
    query, api_key, model, JSON = data
    for i in range(5):
        SambaNova_Client = openai.OpenAI(
                api_key=api_key,
                base_url="https://api.sambanova.ai/v1",
            )

        response = SambaNova_Client.chat.completions.create(
            model=model,
            messages=[{"role":"system","content": "You're a advance data analyst."},{"role":"user","content":query}],
            temperature =  0.1,
            top_p = 0.1,
        )
        if not JSON:
            # print(model)
            return response.choices[0].message.content
        try:
            data = json.loads(extract_query(response.choices[0].message.content))
            return data
        except Exception as e:
            print(e)
            print(response.choices[0].message.content)

final_data = []
with concurrent.futures.ThreadPoolExecutor(max_workers=len(api_keys)) as executor:
        prep = [(i+query, api_keys[m % len(api_keys)], "Meta-Llama-3.1-405B-Instruct", True ) for m,i in enumerate(data)]
        futures = {executor.submit(ask_llm, task): task for task in prep}

        for future in concurrent.futures.as_completed(futures):
            task = futures[future]
            try:
                result = future.result()
                print(result)
                final_data.append(result)
            except Exception as e:
                print(f"Task {task} raised an exception: {e}")


In [10]:
x = split_corpus(str(final_data), max_words=1350)
print(len(x))
# x_data = []
# for m,i in enumerate(x):
#     temp =(i+query, api_keys[m % len(api_keys)], "Meta-Llama-3.1-405B-Instruct", True )
#     res = ask_llm(temp)
#     x_data.append(res)
# x_data


1


In [11]:
y = split_corpus(str(final_data), max_words=1350)
print(len(y))
query = """
                            {data}
                            You have been provided with a report and some images and links related to this question:
                            {question}

                            your job is to unify them and create a Final report that consists of all three of these in MARKDOWN format.
                            Take the images and the link urls from the provided urls only and don't make them up yourself.

                            NOTE: PUT THE IMAGE URLS IN A WAY SO THAT THEY CAN BE DISPLAYED DIRETLY ON THE MARKDOWN.
                                  PUT THE REFERENCE AS YOU SEE FIT.
                                  MAKE THE REPORT LOOK AS HUMAN LIKE AS POSSIBLE.
                            """.format(data = y[0], question = "Jeff Bezos")
y_data = []
for m,i in enumerate(y):
    temp =(query, api_keys[m % len(api_keys)], "Meta-Llama-3.1-405B-Instruct", False )
    res = ask_llm(temp)
    y_data.append(res)

print(y_data[0])

1


# Report: Tesla's Plans for a Van and the Robovan

## Introduction

Tesla has been hinting at developing a van, with mentions of "high passenger-density urban transport" and autonomous buses in its Master Plan Part Deux. Despite Elon Musk's disdain for public transportation, he has listed "bus" and "commercial/passenger vans" as "TBD" in Tesla's Master Plan Part 3.

## Electric Van Market

The electric van market is heating up, with recent releases from Volkswagen, Mercedes, and others, which may prompt Tesla to enter the market.

## Tesla's Robovan

Tesla's Robovan is the surprise of the night, a new van that Elon Musk has dropped hints about in the past. The Robovan has a sleek train engine-like shape and no visible wheels, and can carry up to 20 people or be used to transport goods.

![Tesla Robovan](https://duet-cdn.vox-cdn.com/thumbor/0x0:5662x3184/2400x1600/filters:focal(2831x1592:2832x1593):format(webp)/cdn.vox-cdn.com/uploads/chorus_asset/file/25670518/Robovan_73.jpg)

## Purpo

In [ ]:
print("# Jeff Bezos\n================================\n\nJeff Bezos is a renowned entrepreneur and business magnate, best known as the founder, chairman, and former CEO of Amazon, the world's largest online retailer. He is also the owner of The Washington Post and the founder of Blue Origin, a private aerospace manufacturer and spaceflight services company.\n\n## Early Life and Career\n\nBezos has been a pioneer in the e-commerce industry and has played a significant role in shaping the way people shop and interact with technology. He has also been a prominent figure in the space exploration industry, with Blue Origin aiming to make humanity a multi-planetary species.\n\n## Awards and Recognition\n\nBezos has received numerous awards and accolades for his entrepreneurial achievements and philanthropic efforts. He was awarded the USIBC Global Leadership Award by Indian Prime Minister Narendra Modi in 2016.\n\n![Jeff Bezos at a naturalization ceremony on June 14, 2016](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3e/Naturalization_Ceremony_%2827590969882%29.jpg/220px-Naturalization_Ceremony_%2827590969882%29.jpg)\n\n![Indian Prime Minister Narendra Modi presenting the USIBC Global Leadership Award to Bezos, in Washington, D.C. on June 7, 2016](https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/The_Prime_Minister%2C_Shri_Narendra_Modi_presenting_the_USIBC_Global_Leadership_Award_to_Mr._Jeff_Bezos%2C_in_Washington_DC%2C_USA_on_June_07%2C_2016.jpg/220px-The_Prime_Minister%2C_Shri_Narendra_Modi_presenting_the_USIBC_Global_Leadership_Award_to_Mr._Jeff_Bezos%2C_in_Washington_DC%2C_USA_on_June_07%2C_2016.jpg)\n\n## Philanthropy and Criticism\n\nBezos has been featured in various media outlets and has been recognized for his philanthropic efforts, particularly in the area of early childhood education. However, he has also been criticized for his treatment of employees at Amazon and The Washington Post, as well as his potential conflict of interest with the paper.\n\n![Jeff Bezos in 2017](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c0/Jeff_Bezos_talking.jpg/170px-Jeff_Bezos_talking.jpg)\n\n## Public Presence\n\nBezos has made appearances on various platforms, including TED, C-SPAN, and Charlie Rose.\n\n## References\n\n* [Jeff Bezos Wikipedia page](https://en.wikipedia.org/wiki/Jeff_Bezos)\n* [Amazon official website](https://www.amazon.com)\n* [Blue Origin official website](https://www.blueorigin.com)\n* [The Washington Post official website](https://www.washingtonpost.com)\n* [Jeff Bezos at TED](https://www.ted.com/speakers/jeff_bezos)\n* [Appearances on C-SPAN](https://www.c-span.org/person/?53772)\n* [Jeff Bezos on Charlie Rose](https://charlierose.com/videos/2495)\n* [Jeff Bezos collected news and commentary at The New York Times](http://topics.nytimes.com/top/reference/timestopics/people/b/jeffrey_p_bezos/index.html)\n* [Forbes Profile](https://www.forbes.com/profile/jeff-bezos/)\n\n## Images\n\n![Jeff Bezos visiting Los Angeles Air Force Base](https://upload.wikimedia.org/wikipedia/commons/0/03/Jeff_Bezos_visits_LAAFB_SMC_%283908618%29_%28cropped%29.jpeg)\n\n![Bezos receives the James Smithson Bicentennial medal on June 14, 2016, for his work with Amazon.](//upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Naturalization_Ceremony_%2827590970472%29.jpg/220px-Naturalization_Ceremony_%2827590970472%29.jpg)\n\n![Bezos on October 25, 2017, on his visit to Los Angeles Air Force Base.](//upload.wikimedia.org/wikipedia/commons/thumb/c/c9/Jeff_Bezos-171027-F-DC888008_%2825610066788%29.jpg/170px-Jeff_Bezos-171027-F-DC888008_%2825610066788%29.jpg)\n\n![Wikiquote logo](https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Wikiquote-logo.svg/23px-Wikiquote-logo.svg.png)")

1
# Report: Tesla's Plans for a Van and the Robovan

## Introduction

Tesla has been hinting at developing a van, with mentions of "high passenger-density urban transport" and autonomous buses in its Master Plan Part Deux. Despite Elon Musk's disdain for public transportation, he has listed "bus" and "commercial/passenger vans" as "TBD" in Tesla's Master Plan Part 3.

## Electric Van Market

The electric van market is heating up, with recent releases from Volkswagen, Mercedes, and others, which may prompt Tesla to enter the market.

## Tesla's Robovan

Tesla's Robovan is the surprise of the night, a new van that Elon Musk has dropped hints about in the past. The Robovan has a sleek train engine-like shape and no visible wheels, and can carry up to 20 people or be used to transport goods.

![Tesla Robovan](https://duet-cdn.vox-cdn.com/thumbor/0x0:5662x3184/2400x1600/filters:focal(2831x1592:2832x1593):format(webp)/cdn.vox-cdn.com/uploads/chorus_asset/file/25670518/Robovan_73.jpg)

## Purpose

According to Musk, the Robovan is designed to solve for high density transportation needs.

## References

* [Tesla's Master Plan Part Deux](https://www.tesla.com/blog/master-plan-part-deux)
* [Tesla's Master Plan Part 3](https://www.tesla.com/ns_videos/Tesla-Master-Plan-Part-3.pdf)
* [Elon Musk's disdain for public transportation](https://www.wired.com/story/elon-musk-awkward-dislike-mass-transit/)
* [Tesla](/tesla)
* [Electric Cars](/electric-cars)
* [Cars](/cars)

Note: There is no mention of Jeff Bezos in the provided report, images, and links. The report seems to be related to Elon Musk and Tesla's plans for a van and the Robovan.


In [ ]:
import requests
url = "https://en.wikipedia.org/wiki/Elon_Musk"
res = requests.get(url)
res.text


In [ ]:
res.text

SyntaxError: unterminated string literal (detected at line 43) (477627907.py, line 43)